<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [2]:
import requests
import pandas as pd

In [3]:
toronto_DF = pd.read_csv('Toronto_Neighborhoods.csv')

In [4]:
display(toronto_DF)

,Borough,Postalcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,Moore Park,43.689574,-79.383160
...,...,...,...,...,...
211,York,M6M,Mount Dennis,43.691116,-79.476013
212,York,M6M,Keelsdale and Silverthorn,43.691116,-79.476013
213,York,M6N,Runnymede,43.673185,-79.487262
214,York,M6N,The Junction North,43.673185,-79.487262


In [20]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3cedvIe0HUKfjkoGyFfy91tLazFORcs6XfxpOCTCGx1k='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 2500  # Search radius in meters
category = 11095 # Pest control services category ID: search foursquare categories documentations

# Function to get nearby venues
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
            'categories': category
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighbourhood',
        'Postalcode',
        'Neighbourhood Latitude',
        'Neighbourhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage: Replace with the path to your Toronto Neighborhood CSV file
# Ensure 'toronto_DF' has the columns 'Neighbourhood', 'Postalcode', 'Latitude', and 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighbourhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()

Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park
Processing neighborhood:  Summerhill East
Processing neighborhood: Summerhill West
Processing neighborhood:  Rathnelly
Processing neighborhood:  South Hill
Processing neighborhood:  Forest Hill SE
Processing neighborhood:  Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex
Processing neighborhood:  North Midtown
Processing neighborhood:  Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town
Processing neighborhood:  Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park
Processing neighborhood:  Harbourfront
Processing neighborhood: Garden District
Processing neighborhood:  Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: B

,Neighbourhood,Postalcode,Neighbourhood Latitude,Neighbourhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.728020,-79.388790,Cain Pest & Wildlife Control,59bb5950112c6c44205c90af,43.728194,-79.402771,Pest Control Service
1,Lawrence Park,M4N,43.728020,-79.388790,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service
2,Lawrence Park,M4N,43.728020,-79.388790,Pest and Animal Control Toronto,52851516498ea2013e423f0b,43.711267,-79.396837,Pest Control Service
3,Lawrence Park,M4N,43.728020,-79.388790,AAA Affordable Wildlife Control,274729bde84d49dc9148401e,43.712275,-79.396109,Pest Control Service
4,Davisville North,M4P,43.712751,-79.390197,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service


In [6]:
# The total number of pest control services in Toronto
toronto_venues['Venue'].nunique()

137

In [7]:
# Oberserving the number of competitors in each neighbourhood
venue_counts = toronto_venues.groupby('Neighbourhood')['Venue'].count()
venue_counts

Neighbourhood
 Adelaide            20
 Agincourt North      3
 Albion Gardens       6
 Bathurst Quay        3
 Beaumond Heights     6
                     ..
Willowdale) South     9
Willowdale) West      3
Woodbine Heights     12
York Mills            4
York Mills West       6
Name: Venue, Length: 205, dtype: int64

### Part 2. Interactive leaflet map using coordinate data.


In [8]:
import folium # map rendering library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [9]:
#Create map of all pest control services in Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 12)
# Add markers to map
for lat, lng, name, category, postcode in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['Venue Category'], toronto_venues['Postalcode']):
    label = '{}, {} - {}'.format(name, category, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
display(map_toronto)

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# Pest control services in specific neighbourhood
target = 'Dufferin'

search_area = toronto_venues[toronto_venues['Neighbourhood'] == target]
latitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Latitude']
longitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Longitude']
display(search_area)
map_neighborhood = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighbourhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neighborhood)

map_neighborhood

,Neighbourhood,Postalcode,Neighbourhood Latitude,Neighbourhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
1191,Dufferin,M6H,43.669005,-79.442259,Kind Pest Control,560aeed5498ef4a0fbc7cddd,43.670113,-79.439897,Home Service
1192,Dufferin,M6H,43.669005,-79.442259,Bugs Heat Terminator,027cfef377b746164556f43d,43.672905,-79.443947,Pest Control Service
1193,Dufferin,M6H,43.669005,-79.442259,Toronto Air Soft,5011b02fe4b037378be1f1b6,43.670398,-79.435219,Casino
1194,Dufferin,M6H,43.669005,-79.442259,Pest Control Toronto,57715064498ec3ff9146ab02,43.665764,-79.450236,Home Service
1195,Dufferin,M6H,43.669005,-79.442259,Pest Control Toronto Team,5362917211d25710fe57f69e,43.661248,-79.443665,Pest Control Service
1196,Dufferin,M6H,43.669005,-79.442259,Bed Bug Exterminator Pro,57c7076f498e500d301f8228,43.656958,-79.448588,Pest Control Service
1197,Dufferin,M6H,43.669005,-79.442259,Merkem Pest Solutions,589bd7aa0bc55b5f8ef35448,43.661951,-79.424793,Home Service
1198,Dufferin,M6H,43.669005,-79.442259,SWAT Wildlife,4f8e92e7e4b0036b6f308fd9,43.654321,-79.440122,Pest Control Service
1199,Dufferin,M6H,43.669005,-79.442259,Quest Pest Management,9d648bfa2f4a4bf41ed7d037,43.671627,-79.420683,Pest Control Service
1200,Dufferin,M6H,43.669005,-79.442259,Pest Control Toronto .ca,9c2c9d2221964df80a233a2b,43.671633,-79.420590,Pest Control Service


/opt/conda/lib/python3.11/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [12]:
# a function to loop through the list of pest control services in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"


        # Make the GET request
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips


nearby_tips = getTips(
    venues=search_area['Venue'],
    fsq_ids=search_area['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Kind Pest Control
Processing venue: Bugs Heat Terminator
Processing venue: Toronto Air Soft
Processing venue: Pest Control Toronto
Processing venue: Pest Control Toronto Team
Processing venue: Bed Bug Exterminator Pro
Processing venue: Merkem Pest Solutions
Processing venue: SWAT Wildlife
Processing venue: Quest Pest Management
Processing venue: Pest Control Toronto .ca
Processing venue: Exterminex Pest Control Svc
Processing venue: Pest Control Toronto Exterminator
Processing venue: Allied Pest Control
Processing venue: Toronto Bed Bug .ca
Failed to get data for Toronto Bed Bug .ca. Status code: 404
Error message: invalid place specified: 54fcd7f7010745bf03e185e8


,venue_name,id,created_at,text
0,1191 Kind Pest Control 1192...,589e0c52b3cdc84d12734ae1,2017-02-10T18:54:10.000Z,افضل شركة مكافحة حشرات بالرياض مع الضمان باستخ...


In [13]:
# Function to get place details
def get_place_details(fsq_id):
    url = f"https://api.foursquare.com/v3/places/{fsq_id}"
    headers = {
        "Accept": "application/json",
        "Authorization": API_KEY
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to get details for fsq_id {fsq_id}. Status code: {response.status_code}")
        return None
    
    return response.json()

# Function to extract details
def extract_details(fsq_id):
    details = get_place_details(fsq_id)
    if details is None:
        return None, None  # Return None if the API call fails
    
    rating = details.get("rating", None)  # Retrieve the rating
    price_level = details.get("price", {}).get("tier", None)  # Retrieve the price tier (if available)
    popularity = details.get("popularity", None)  # Retrieve popularity if available
    return rating, price_level, popularity

In [14]:
# Placeholder columns for ratings and prices
# toronto_venues['Rating'] = None
# toronto_venues['Price Level'] = None
# toronto_venues['Popularity'] = None

# Iterate over each fsq_id and fetch details
for idx, fsq_id in toronto_venues['fsq_id'].items():
    print(f"Fetching details for fsq_id: {fsq_id}")
    rating, price_level, popularity = extract_details(fsq_id)
    toronto_venues.at[idx, 'Rating'] = rating
    toronto_venues.at[idx, 'Price Level'] = price_level
    toronto_venues.at[idx, 'Popularity'] = popularity

Fetching details for fsq_id: 59bb5950112c6c44205c90af
Fetching details for fsq_id: 049ad9a78fbc4226a4e3142d
Fetching details for fsq_id: 52851516498ea2013e423f0b
Fetching details for fsq_id: 274729bde84d49dc9148401e
Fetching details for fsq_id: 049ad9a78fbc4226a4e3142d
Fetching details for fsq_id: 52851516498ea2013e423f0b
Fetching details for fsq_id: 662c6f812eda4bddd7a5f6f9
Fetching details for fsq_id: 59bb5950112c6c44205c90af
Fetching details for fsq_id: 3aa18c9e6aeb4ca69f5dc5ab
Fetching details for fsq_id: 274729bde84d49dc9148401e
Fetching details for fsq_id: 52851516498ea2013e423f0b
Fetching details for fsq_id: 049ad9a78fbc4226a4e3142d
Fetching details for fsq_id: 59bb5950112c6c44205c90af
Fetching details for fsq_id: 5d1f51fa7dc742002321b364
Fetching details for fsq_id: 5f2256ed54f23d538126dd4d
Fetching details for fsq_id: 662c6f812eda4bddd7a5f6f9
Fetching details for fsq_id: 274729bde84d49dc9148401e
Fetching details for fsq_id: 049ad9a78fbc4226a4e3142d
Fetching details for fsq_id:

In [15]:
toronto_venues.head()

,Neighbourhood,Postalcode,Neighbourhood Latitude,Neighbourhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,Rating,Price Level,Popularity
0,Lawrence Park,M4N,43.728020,-79.388790,Cain Pest & Wildlife Control,59bb5950112c6c44205c90af,43.728194,-79.402771,Pest Control Service,None,None,None
1,Lawrence Park,M4N,43.728020,-79.388790,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service,None,None,None
2,Lawrence Park,M4N,43.728020,-79.388790,Pest and Animal Control Toronto,52851516498ea2013e423f0b,43.711267,-79.396837,Pest Control Service,None,None,None
3,Lawrence Park,M4N,43.728020,-79.388790,AAA Affordable Wildlife Control,274729bde84d49dc9148401e,43.712275,-79.396109,Pest Control Service,None,None,None
4,Davisville North,M4P,43.712751,-79.390197,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service,None,None,None


In [16]:
toronto_venues.to_csv("toronto_venues.csv", index=False)

### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  

In [ ]:
# since we have no data on our competitors available in Foursquare, we'll need to take a different approach.
# we will be gathering data on our commercial demographic (not just residential as in Part 2) to further justify
# our choice of neighbourhood

In [23]:
# location of grocery stores
categoryIDs = [17069, 15014, 11058] # Category IDs for 'Grocery Store', 'Hospital', 'Food Distribution Center'

for i in range(len(categoryIDs)):
    category = categoryIDs[i]
    grocery_stores = getNearbyVenues(
        names=toronto_DF['Neighbourhood'],
        postalcode=toronto_DF['Postalcode'],
        latitudes=toronto_DF['Latitude'],
        longitudes=toronto_DF['Longitude'],
        radius=radius
)

grocery_stores.head()

Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park
Processing neighborhood:  Summerhill East
Processing neighborhood: Summerhill West
Processing neighborhood:  Rathnelly
Processing neighborhood:  South Hill
Processing neighborhood:  Forest Hill SE
Processing neighborhood:  Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex
Processing neighborhood:  North Midtown
Processing neighborhood:  Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town
Processing neighborhood:  Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park
Processing neighborhood:  Harbourfront
Processing neighborhood: Garden District
Processing neighborhood:  Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: B

,Neighbourhood,Postalcode,Neighbourhood Latitude,Neighbourhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Davisville North,M4P,43.712751,-79.390197,Barclay MacDonald Co Inc,d8c2a73a463e43775b27a4a6,43.705174,-79.374756,Automotive Repair Shop
1,Davisville,M4S,43.704324,-79.388790,Barclay MacDonald Co Inc,d8c2a73a463e43775b27a4a6,43.705174,-79.374756,Automotive Repair Shop
2,Moore Park,M4T,43.689574,-79.383160,Barclay MacDonald Co Inc,d8c2a73a463e43775b27a4a6,43.705174,-79.374756,Automotive Repair Shop
3,Summerhill East,M4T,43.689574,-79.383160,Barclay MacDonald Co Inc,d8c2a73a463e43775b27a4a6,43.705174,-79.374756,Automotive Repair Shop
4,The Annex,M5R,43.672710,-79.405678,D and S Catering Service,eecea9e79de4445ececa03e5,43.660914,-79.382683,Food Distribution Center


In [ ]:
# map of categoryIDs based on neighbourhood
